In [21]:
import cv2;
import numpy as np; np.random.seed(7);
import os;
import sys;
import tensorflow.keras as keras
import keras.backend as K;
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator

In [22]:
def unet(input_size = (128,128,1)):
     inputs = Input(input_size) #128x128
     conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same' )(inputs)
     b1 = BatchNormalization()(conv2)
     conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same' )(b1)
     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #64x64
     conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same' )(pool2)
     b2 = BatchNormalization()(conv3)
     conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same' )(conv3)
     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3) #32x32
     conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same' )(pool3)
     b3 = BatchNormalization()(conv4)
     conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same' )(b3)
     drop4 = Dropout(0.5)(conv4) #32x32
     pool4 = MaxPooling2D(pool_size=(2, 2))(drop4) #16x16
     conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same' )(pool4)
     b4 = BatchNormalization()(conv5)
     conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same' )(b4)
     drop5 = Dropout(0.5)(conv5) #16x16
     up6 = Conv2D(256, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(drop5)) #32x32
     merge6 = concatenate([drop4,up6], axis = 3)#32x32
     conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same' )(merge6)
     b5 = BatchNormalization()(conv6)
     conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same' )(b5)
     up7 = Conv2D(128, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv6)) #64x64
        
     merge7 = concatenate([conv3,up7], axis = 3)
     conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same' )(merge7)
     b6 = BatchNormalization()(conv7)
     conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same' )(b6)
     up8 = Conv2D(164, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv7)) #128x128
     merge8 = concatenate([conv2,up8], axis = 3)
     conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same' )(merge8)
     b7 = BatchNormalization()(conv8)
     conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same' )(b7)
     conv8 = Conv2D(2, 3, activation = 'relu', padding = 'same' )(conv8)
     b8 = BatchNormalization()(conv8)
     conv9 = Conv2D(1, 1, activation = 'sigmoid', padding = 'same', bias_initializer=keras.initializers.Constant(value=1.5))(b8)
     model = Model(inputs = inputs, outputs = conv9)
     model.compile(optimizer = Adam(lr = 1e-3),
     loss = 'mean_squared_error', metrics = ['accuracy'])
     model.summary()
     return model

In [23]:
nomeprog="membrane_unet"
train_path='/home/lucas/Desktop/poli/PSI3472/aula8e9/membrane/train/';
outDir = "/home/lucas/Desktop/poli/PSI3472/aula8e9/membrane_out"; os.chdir(outDir)

In [24]:
aug_dict = dict(rotation_range=10, #Int. Degree range for random rotations.
 width_shift_range=0.05, #float: fraction of total width, if < 1, or pixels if >= 1.
 height_shift_range=0.05, #float: fraction of total height, if < 1, or pixels if >= 1.
 shear_range=10, #Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees)
 zoom_range=0.2, #Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range].
 horizontal_flip=False, #Boolean. Randomly flip inputs horizontally.
 fill_mode='reflect'); #One of {"constant", "nearest", "reflect" or "wrap"}.

In [25]:
image_folder='image';
mask_folder= 'label';
target_size = (128,128);
batch_size=10;
seed = 7;
save_to_dir = None;

In [26]:
image_datagen = ImageDataGenerator(**aug_dict);
mask_datagen = ImageDataGenerator(**aug_dict);

In [27]:
image_generator = image_datagen.flow_from_directory(
 train_path,
 classes = [image_folder],
 class_mode = None,
 color_mode = "grayscale",
 target_size = target_size,
batch_size = batch_size,
 seed = seed);

Found 30 images belonging to 1 classes.


In [28]:
mask_generator = mask_datagen.flow_from_directory(
 train_path,
 classes = [mask_folder],
 class_mode = None,
 color_mode = "grayscale",
 target_size = target_size,
 batch_size = batch_size,
 seed = seed);

Found 30 images belonging to 1 classes.


In [29]:
def trainGenerator():
 train_generator=zip(image_generator,mask_generator)
 for (img,mask) in train_generator:
     img=img/255
     mask=mask/255
     mask[mask > 0.5] = 1
     mask[mask <= 0.5] = 0
     yield(img,mask)

In [30]:
#<<<<<<<<<<<<<<<<< main <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#Escolha entre comecar treino do zero ou continuar o treino de onde parou
model = unet();
#model = load_model(nomeprog+".h5");
model.fit(trainGenerator(),steps_per_epoch=30,epochs=10,verbose=2);
model.save(nomeprog+".h5");

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 128, 128, 64) 640         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 128, 128, 64) 256         conv2d_38[0][0]                  
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 128, 128, 64) 36928       batch_normalization_6[0][0]      
_______________________________________________________________________________________

30/30 - 238s - loss: 0.1181 - accuracy: 0.8300
Epoch 2/10
30/30 - 219s - loss: 0.0908 - accuracy: 0.8774
Epoch 3/10
30/30 - 227s - loss: 0.0819 - accuracy: 0.8907
Epoch 4/10
30/30 - 253s - loss: 0.0769 - accuracy: 0.8974
Epoch 5/10
30/30 - 227s - loss: 0.0726 - accuracy: 0.9030
Epoch 6/10
30/30 - 223s - loss: 0.0701 - accuracy: 0.9057
Epoch 7/10
30/30 - 235s - loss: 0.0680 - accuracy: 0.9081
Epoch 8/10
30/30 - 223s - loss: 0.0655 - accuracy: 0.9113
Epoch 9/10
30/30 - 231s - loss: 0.0630 - accuracy: 0.9146
Epoch 10/10
30/30 - 275s - loss: 0.0618 - accuracy: 0.9160


# Saída sem Batch Normalization

Epoch 10/10
30/30 - 260s - loss: 0.0879 - accuracy: 0.8927


# Saída com Batch Normalization

Epoch 10/10
30/30 - 275s - loss: 0.0618 - accuracy: 0.9160